In [ ]:
!pip install ultralytics

In [ ]:
### MAPPING ISIC - 2019

import os
import pandas as pd
import shutil

# 1) Paths – adjust if needed
csv_path    = '/kaggle/input/original-isic-2019/ISIC_2019_Test_GroundTruth.csv'
input_dir   = '/kaggle/input/annotated-isic-2019-test/runs/detect/yolov11_detections/crops/original-isic-2019/ISIC_2019_Test_Input/ISIC_2019_Test_Input'
output_dir  = '/kaggle/working/ISIC_2019_Test_By_Class'

# 2) Load CSV & build image→class map
df = pd.read_csv(csv_path)
classes = ['MEL','NV','BCC','AK','BKL','DF','VASC','SCC','UNK']

# build a dict: { 'ISIC_0000002': 'MEL', … }
img2cls = {}
for _, row in df.iterrows():
    img_id = row['image']
    # find which class has 1.0
    for c in classes:
        if row[c] == 1.0:
            img2cls[img_id] = c
            break

# 3) Make output folders
for c in classes:
    os.makedirs(os.path.join(output_dir, c), exist_ok=True)

# 4) Iterate through all cropped images
for fname in os.listdir(input_dir):
    if not fname.lower().endswith('.jpg'):
        continue

    # e.g. fname = "ISIC_0000002_1.jpg"
    stem = os.path.splitext(fname)[0]           # "ISIC_0000002_1"
    orig = stem.rsplit('_', 1)[0]               # "ISIC_0000002"

    cls = img2cls.get(orig, 'UNK')              # default to UNK if missing
    src = os.path.join(input_dir, fname)
    dst = os.path.join(output_dir, cls, fname)

    shutil.copy(src, dst)

print("Done! Crops have been sorted into:", output_dir)